# Supervised Baselines

In [1]:
import ast
import numpy as np
import pandas as pd
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, GlobalMaxPooling1D, Input, concatenate
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Lambda, Embedding
from keras import regularizers, optimizers
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from corpus_WiNER.corpus_utils import *
from gensim.models import KeyedVectors

Using TensorFlow backend.


# MLP 

Utilizando la estrategia de decaimiento exponencial.

### Cargamos los datos pre-procesados y los filtramos para obtener:

- 100000 instancias de train
- &nbsp; 20000 instancias de dev
- &nbsp; 20000 instancias de test

In [2]:
word_vectors = np.load('./corpus_WiNER/word_vectors/wv_train_exp_decay_W_5.npz')
entity_vector = np.load('./corpus_WiNER/entity_vectors/ev_train_exp_decay_W_5.npz')
X_train = word_vectors.items()[0][1][:100000]
y_train = entity_vector.items()[0][1][:100000]
word_vectors = np.load('./corpus_WiNER/word_vectors/wv_dev_exp_decay_W_5.npz')
entity_vector = np.load('./corpus_WiNER/entity_vectors/ev_dev_exp_decay_W_5.npz')
X_dev = word_vectors.items()[0][1][:20000]
y_dev = entity_vector.items()[0][1][:20000]
word_vectors = np.load('./corpus_WiNER/word_vectors/wv_test_exp_decay_W_5.npz')
entity_vector = np.load('./corpus_WiNER/entity_vectors/ev_test_exp_decay_W_5.npz')
X_test = word_vectors.items()[0][1][:20000]
y_test = entity_vector.items()[0][1][:20000]

In [3]:
print('# word vectors:', len(X_train))
print('# non entities', list(y_train).count('O'))
print('# word vectors:', len(X_dev))
print('# non entities', list(y_dev).count('O'))
print('# word vectors:', len(X_test))
print('# non entities', list(y_test).count('O'))

# word vectors: 100000
# non entities 55825
# word vectors: 20000
# non entities 9634
# word vectors: 20000
# non entities 10410


In [4]:
y_train = [tagToInt(y) for y in y_train]
y_dev = [tagToInt(y) for y in y_dev]
y_test = [tagToInt(y) for y in y_test]
y_train[:10] # this transformation is needed to apply to_categorical() keras method

[0, 0, 0, 0, 2, 2, 2, 1, 1, 1]

In [5]:
num_classes = 5 # PER - LOC - ORG - MISC - O
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_dev = to_categorical(y_dev, num_classes)
y_test = to_categorical(y_test, num_classes)

In [16]:
# # Función que genera los modelos y que se usara en la grilla de validación cruzada.
# def build_model2(nodes1 = 100, nodes2 = 200, lr = 0.001, 
#                 l2 = 0.01, drop = 0.1):
nodes1 = 300
nodes2 = 512
lr = 0.001
l2 = 0.01
drop = 0.1

model = Sequential()    
model.add(Dense(nodes1,
                input_shape=(300,),
                activation = 'relu',
                kernel_regularizer = regularizers.l2(l2)
         )
)      
model.add(Dropout(drop))   

model.add(Dense(nodes2,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(l2)))
model.add(Dense(nodes2,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(l2)))

model.add(Dense(256,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(l2)))
model.add(Dense(256,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(l2)))


model.add(Dropout(drop))
model.add(Dense(128,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(l2)))
model.add(Dense(5, activation = 'softmax')) # PER - LOC - ORG - MISC - O

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               154112    
_________________________________________________________________
dense_27 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_28 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_29 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
__________

In [18]:
model.compile(optimizer = optimizers.Adadelta(lr = lr),
              loss = categorical_crossentropy,
              metrics = ['accuracy'])

In [19]:
batch_size = 512
epochs = 10

In [20]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_dev, y_dev))

Train on 100000 samples, validate on 20000 samples
Epoch 1/10
100000/100000 [==============================] - 4s 43us/step - loss: 21.1002 - acc: 0.1901 - val_loss: 21.0348 - val_acc: 0.3146
Epoch 2/10
100000/100000 [==============================] - 3s 34us/step - loss: 20.9520 - acc: 0.4330 - val_loss: 20.8860 - val_acc: 0.4757
Epoch 3/10
100000/100000 [==============================] - 3s 35us/step - loss: 20.7924 - acc: 0.5508 - val_loss: 20.7286 - val_acc: 0.4817
Epoch 4/10
100000/100000 [==============================] - 3s 34us/step - loss: 20.6237 - acc: 0.5582 - val_loss: 20.5644 - val_acc: 0.4817
Epoch 5/10
100000/100000 [==============================] - 3s 33us/step - loss: 20.4466 - acc: 0.5582 - val_loss: 20.3952 - val_acc: 0.4817
Epoch 6/10
100000/100000 [==============================] - 3s 34us/step - loss: 20.2633 - acc: 0.5582 - val_loss: 20.2245 - val_acc: 0.4817
Epoch 7/10
100000/100000 [==============================] - 3s 34us/step - loss: 20.0786 - acc: 0.5583 

# CNN 

Utilizando ventana simétrica de palabras que rodea a la objetivo.

### Cargamos los datos pre-procesados y los filtramos para obtener:

- 100000 instancias de train
- &nbsp; 20000 instancias de dev
- &nbsp; 20000 instancias de test

In [2]:
data_train = pd.read_csv('./corpus_WiNER/cnn_instances/words_entity_W_2_cnn_train.csv')
data_dev = pd.read_csv('./corpus_WiNER/cnn_instances/words_entity_W_2_cnn_dev.csv')
data_test = pd.read_csv('./corpus_WiNER/cnn_instances/words_entity_W_2_cnn_test.csv')
data_train.head()

,words,entityType
0,"['', '', 'Watching', 'Ellie', 'is']",MISC
1,"['', 'Watching', 'Ellie', 'is', 'an']",MISC
2,"['Watching', 'Ellie', 'is', 'an', 'American']",O
3,"['is', 'an', 'American', 'sitcom', 'that']",LOC
4,"['that', 'starred', 'Julia', 'Louis-Dreyfus', ...",PER


In [3]:
X_train = data_train['words'].values[:100000]
y_train = data_train['entityType'].values[:100000]
X_dev = data_dev['words'].values[:20000]
y_dev = data_dev['entityType'].values[:20000]
X_test = data_test['words'].values[:20000]
y_test = data_test['entityType'].values[:20000]

In [4]:
print('# instances:', len(X_train))
print('# non entities', list(y_train).count('O'))
print('# instances:', len(X_dev))
print('# non entities', list(y_dev).count('O'))
print('# instances:', len(X_test))
print('# non entities', list(y_test).count('O'))

# instances: 100000
# non entities 54508
# instances: 20000
# non entities 9590
# instances: 20000
# non entities 10856


In [5]:
y_train = [tagToInt(y) for y in y_train]
y_dev = [tagToInt(y) for y in y_dev]
y_test = [tagToInt(y) for y in y_test]
y_train[:10] # this transformation is needed to apply to_categorical() keras method

[3, 3, 4, 1, 0, 0, 0, 0, 4, 2]

In [6]:
def transform_input(instances, mapping):
    """Replaces the words in instances with their index in mapping.
    Args:
        instances: a list of text instances.
        mapping: an dictionary from words to indices.
    Returns:
        A matrix with shape (n_instances, m_words)."""
    word_indices = []
    for instance in instances:
        l = []
        for word in ast.literal_eval(instance):
            try:
                l.append(mapping[word].index)
            except KeyError:
                l.append(0) # index to '</s>' word vector
        word_indices.append(l)
        
    return word_indices

In [ ]:
w2v_model = KeyedVectors.load('./models/google/word2vecGoogle.model')

In [ ]:
X_train = transform_input(X_train, w2v_model.vocab)
X_dev = transform_input(X_dev, w2v_model.vocab)
X_test = transform_input(X_test, w2v_model.vocab)

In [ ]:
batch_size = 512  # For mini-batch gradient descent
num_classes = 5 # PER - LOC - ORG - MISC - O
epochs = 10
len_words = 5
input_size = len_words # amount of words by row
train_examples = len(X_train)
test_examples = len(X_test)
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_dev = to_categorical(y_dev, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
# https://stackoverflow.com/questions/43396572/dimension-of-shape-in-conv1d
steps = 5 # number of words in the sentence
channels = 1
input_shape = (steps, channels) #3D tensor with shape: `(batch, steps, channels)`
# # Output shape
#     3D tensor with shape: `(batch, new_steps, filters)`
X_train = np.asarray(X_train)
X_dev = np.asarray(X_dev)
X_test = np.asarray(X_test)

In [ ]:
X_train.shape

In [ ]:
X_train[:5]

### CNN Model

In [ ]:
conv_filters = 10
pool_size = 2
inp = Input(shape=(X_train.shape[1],))
print(inp.shape)

In [ ]:
emb = Embedding(len(w2v_model.vocab),  # Vocabulary size
                w2v_model.vector_size, # Embedding size
                weights=[w2v_model.vectors], # Word vectors
                trainable=False  # This indicates the word vectors must not be changed
                                 # during training.
      )(inp)
print(emb.shape)
# The output here has shape (batch_size (?), words_in_reviews (?), embedding_size)

In [ ]:
# Input shape
#     3D tensor with shape: `(batch, steps, channels)`

# Output shape
#     3D tensor with shape: `(batch, new_steps, filters)`
#     `steps` value might have changed due to padding or strides.

# Specify each convolution layer and their kernel size i.e. n-grams 
conv1_1 = Conv1D(filters=conv_filters, kernel_size=2, activation='relu')(emb)
btch1_1 = BatchNormalization()(conv1_1)
maxp1_1 = MaxPooling1D(pool_size=pool_size)(btch1_1)
flat1_1 = Flatten()(maxp1_1)

conv1_2 = Conv1D(filters=conv_filters, kernel_size=3, activation='relu')(emb)
btch1_2 = BatchNormalization()(conv1_2)
maxp1_2 = MaxPooling1D(pool_size=pool_size)(btch1_2)
flat1_2 = Flatten()(maxp1_2)

conv1_3 = Conv1D(filters=conv_filters, kernel_size=4, activation='relu')(emb)
btch1_3 = BatchNormalization()(conv1_3)
maxp1_3 = MaxPooling1D(pool_size=pool_size)(btch1_3)
flat1_3 = Flatten()(maxp1_3)

# Gather all convolution layers
cnct = concatenate([flat1_1, flat1_2, flat1_3], axis=1)
drp1 = Dropout(0)(cnct)

dns1  = Dense(128, activation='relu')(drp1)
out = Dense(num_classes, activation='softmax')(dns1)

In [ ]:
model = Model(inputs=inp, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_dev, y_dev))